### 국회의원 정보 스크래핑
* 국회의원이름. 국회의원 id 추출
* 국회의원 상세 페이지에 300번 요청을 보내서 상세 정보 추출
* pandas DataFrame에 저장
* 시각화(막대그래프, 히스토그램, 파이차트)
* DB에 members 테이블에 저장

In [3]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?currentPage=1&rowPerPage=300'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

atag_list = soup.select('.memberna_list dl dt a')
# print(len(atag_list))

id_list = list()
for idx, atag in enumerate(atag_list,1):
    name = atag.text
    link = atag['href']
    matched = re.search(r"(\d+)",link)
#     print(matched.group(0),matched.group(1))
    members_id = matched.group(0)
#     print(idx, name, members_id)
    id_list.append(members_id)
id_list
    

['9771230',
 '9771142',
 '9771174',
 '9771233',
 '9771283',
 '9770933',
 '9771116',
 '9771276',
 '9771168',
 '9771007',
 '9771109',
 '9771180',
 '9770931',
 '9770956',
 '9771224',
 '9771165',
 '9770679',
 '9771108',
 '9770977',
 '9771259',
 '9770998',
 '9770927',
 '9771258',
 '9770712',
 '9771152',
 '9771164',
 '9771181',
 '9770719',
 '9770997',
 '9771136',
 '9770723',
 '9771120',
 '9771173',
 '9770941',
 '9770984',
 '9771285',
 '9771268',
 '9770725',
 '9770527',
 '9771024',
 '9771195',
 '9770985',
 '9771208',
 '9771089',
 '9771210',
 '9771220',
 '9771144',
 '9771169',
 '9771111',
 '9771225',
 '9770975',
 '9770981',
 '9770934',
 '9771249',
 '9771186',
 '9771147',
 '9771127',
 '9771214',
 '9771206',
 '9771171',
 '9771023',
 '9771090',
 '9771009',
 '9771192',
 '9771281',
 '9770982',
 '9770986',
 '9770741',
 '9771237',
 '9770742',
 '9770991',
 '9771223',
 '9771270',
 '9771216',
 '9771134',
 '9771228',
 '9770752',
 '9770753',
 '9770755',
 '9771141',
 '9771274',
 '9771091',
 '9771187',
 '97

In [4]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

print('----------작업시작----------')

member_list = []
for idx, id in enumerate(id_list,1):
    detail_url = f'https://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd={id}'
#     print(idx, detail_url)
    html = requests.get(detail_url).text
    soup = BeautifulSoup(html, 'html.parser')
    # 국회의원 한명의 상세정보를 저장하는 dictionary
    member_dict = {}
    # dt tag
    dt_list = []
    for dt_tag in soup.select('.info_mna .pro_detail dt'):
        dt_list.append(dt_tag.text)
#     print(dt_list) 
    # dd tag
    dd_list = []
    for dd_tag in soup.select('.info_mna .pro_detail dd'):
        regex = re.compile(r'[\n\r\t]')
        result = regex.sub("",dd_tag.text.strip()).replace(" ","")
        dd_list.append(result)
        
    member_dict = dict(zip(dt_list,dd_list))
#     print(member_dict) 
    
    for div_tag in soup.select('.info_mna .profile'):
#         print(div_tag.text)
        member_dict['이름'] = div_tag.select('h4')[0].text
        img_url = div_tag.find('img').attrs['src']
        member_dict['이미지'] = urljoin(detail_url,img_url)
#         print(div_tag)
        member_dict['생년월일'] = div_tag.select_one("li:nth-of-type(4)").text
#         print(birth_data)
    member_list.append(member_dict)
# member_list

print(len(member_list))
print('----------작업종료----------')

----------작업시작----------
300
----------작업종료----------


In [6]:
member_list[298:]

[{'정당': '더불어민주당',
  '선거구': '대전중구',
  '소속위원회': '산업통상자원중소벤처기업위원회,예산결산특별위원회',
  '당선횟수': '초선(21대)',
  '사무실전화': '02-784-1935',
  '홈페이지': '',
  '이메일': 'unhah0301@gmail.com',
  '보좌관': '박승규,방인호',
  '비서관': '장진섭,최형준',
  '비서': '유혜정,홍승경,이제문,정윤주,정서원',
  '취미, 특기': '',
  '이름': '황운하',
  '이미지': 'https://www.assembly.go.kr/photo/9771161.jpg',
  '생년월일': '1962-09-10'},
 {'정당': '더불어민주당',
  '선거구': '서울양천구갑',
  '소속위원회': '국방위원회',
  '당선횟수': '재선(20대,21대)',
  '사무실전화': '02-784-8551',
  '홈페이지': 'http://blog.naver.com/hwanghee67',
  '이메일': 'hwanghee6728@gmail.com',
  '보좌관': '나성채,이용협',
  '비서관': '김순구,김진하',
  '비서': '이정수,이종서,장유리,강조원,나종호',
  '취미, 특기': '',
  '이름': '황희',
  '이미지': 'https://www.assembly.go.kr/photo/9770936.jpg',
  '생년월일': '1967-07-28'}]

In [12]:
#[{},{},{}]
# {} -> Series (1개의 Row)
# [] -> DataFrame

import pandas as pd

data_df = pd.DataFrame(columns=['이름','이미지','생년월일','정당','선거구','소속위원회','당선횟수','사무실전화','홈페이지','이메일','보좌관','비서관','비서','취미, 특기'])
for member in member_list:
    series_ob = pd.Series(member)
    data_df = data_df.append(series_ob,ignore_index=True)
data_df


,이름,이미지,생년월일,정당,선거구,소속위원회,당선횟수,사무실전화,홈페이지,이메일,보좌관,비서관,비서,"취미, 특기"
0,강기윤,https://www.assembly.go.kr/photo/9771230.jpg,1960-06-04,미래통합당,경남창원시성산구,보건복지위원회,"재선(19대,21대)",02-784-1751,http://blog.naver.com/ggotop,ggotop@naver.com,"강종길,김홍광","박응서,최광림","김영록,안효상,이유진,홍지형,김지훈",
1,강대식,https://www.assembly.go.kr/photo/9771142.jpg,1959-11-02,미래통합당,대구동구을,국방위원회,초선(21대),,,,"박홍규,정운태",유진영,"박종원,박순권,김광연,김현정,송민욱",
2,강득구,https://www.assembly.go.kr/photo/9771174.jpg,1963-05-27,더불어민주당,경기안양시만안구,교육위원회,초선(21대),02-784-2747~9,https://blog.naver.com/dulipapa,mainsail440@daum.net,유진우,"김도균,홍미하","최경순,김혜원,최기섭,조나연,오세령",
3,강민국,https://www.assembly.go.kr/photo/9771233.jpg,1971-03-03,미래통합당,경남진주시을,정무위원회,초선(21대),,,,"강민승,박진우","신우성,오경훈","성환종,김민규,김오주,정재희,이승연",
4,강민정,https://www.assembly.go.kr/photo/9771283.jpg,1961-04-26,열린민주당,비례대표,교육위원회,초선(21대),,,,전경원,"김민혜,윤호숙","김수안,김성용,민지홍,서지혜,이태경",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,홍정민,https://www.assembly.go.kr/photo/9771183.jpg,1978-11-24,더불어민주당,경기고양시병,"국회운영위원회,과학기술정보방송통신위원회",초선(21대),,,,"유지영,조성환","이용남,주영민","오미숙,한상득,윤한균,박희철,박지혜",
296,홍준표,https://www.assembly.go.kr/photo/9771145.jpg,1954-12-05,무소속,대구수성구을,"국방위원회,예산결산특별위원회","5선(15대,16대,17대,18대,21대)","02-784-7610,02-6788-7481",https://blog.naver.com/jphong803,jphong803@naver.com,"김민준,이종헌","김윤환,손성호","양재유,김은영,고천석,권희연,김태형",
297,황보승희,https://www.assembly.go.kr/photo/9771130.jpg,1976-08-05,미래통합당,부산중구영도구,"과학기술정보방송통신위원회,예산결산특별위원회",초선(21대),,,,"서덕교,이준우","고범욱,홍영민","최지현,정인호,홍주형,윤성환,서승환",
298,황운하,https://www.assembly.go.kr/photo/9771161.jpg,1962-09-10,더불어민주당,대전중구,"산업통상자원중소벤처기업위원회,예산결산특별위원회",초선(21대),02-784-1935,,unhah0301@gmail.com,"박승규,방인호","장진섭,최형준","유혜정,홍승경,이제문,정윤주,정서원",


In [13]:
print(data_df.columns)
print(data_df.index)
print(data_df.shape)

Index(['이름', '이미지', '생년월일', '정당', '선거구', '소속위원회', '당선횟수', '사무실전화', '홈페이지',
       '이메일', '보좌관', '비서관', '비서', '취미, 특기'],
      dtype='object')
RangeIndex(start=0, stop=300, step=1)
(300, 14)


In [15]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이름      300 non-null    object
 1   이미지     300 non-null    object
 2   생년월일    300 non-null    object
 3   정당      300 non-null    object
 4   선거구     300 non-null    object
 5   소속위원회   300 non-null    object
 6   당선횟수    300 non-null    object
 7   사무실전화   300 non-null    object
 8   홈페이지    300 non-null    object
 9   이메일     300 non-null    object
 10  보좌관     300 non-null    object
 11  비서관     300 non-null    object
 12  비서      300 non-null    object
 13  취미, 특기  300 non-null    object
dtypes: object(14)
memory usage: 32.9+ KB


In [21]:
data_df['정당'].value_counts()

더불어민주당    176
미래통합당     103
무소속         7
정의당         6
국민의당        3
열린민주당       3
시대전환        1
기본소득당       1
Name: 정당, dtype: int64

In [23]:
data_df['정당'].unique()

array(['미래통합당', '더불어민주당', '열린민주당', '정의당', '무소속', '국민의당', '기본소득당', '시대전환'],
      dtype=object)

In [30]:
data_df['선거구'].unique()[0:4]
data_df['당선횟수'].unique()
data_df['소속위원회'].unique()

array(['보건복지위원회', '국방위원회', '교육위원회', '정무위원회', '국회운영위원회,보건복지위원회', '환경노동위원회',
       '국토교통위원회', '산업통상자원중소벤처기업위원회', '기획재정위원회', '국회운영위원회,교육위원회',
       '농림축산식품해양수산위원회', '행정안전위원회', '교육위원회,여성가족위원회', '기획재정위원회,정보위원회',
       '외교통일위원회,예산결산특별위원회', '법제사법위원회', '국회운영위원회,법제사법위원회',
       '보건복지위원회,여성가족위원회', '행정안전위원회,예산결산특별위원회', '국방위원회,정보위원회',
       '과학기술정보방송통신위원회', '외교통일위원회', '국회운영위원회,환경노동위원회', '문화체육관광위원회',
       '국회운영위원회,행정안전위원회', '국회운영위원회,농림축산식품해양수산위원회', '보건복지위원회,예산결산특별위원회',
       '국회운영위원회,산업통상자원중소벤처기업위원회,여성가족위원회', '교육위원회,예산결산특별위원회',
       '국회운영위원회,외교통일위원회,정보위원회', '국회운영위원회,기획재정위원회', '정무위원회,예산결산특별위원회',
       '외교통일위원회,정보위원회', '국회운영위원회,국토교통위원회', '환경노동위원회,정보위원회',
       '국회운영위원회,과학기술정보방송통신위원회', '', '국회운영위원회,국방위원회', '기획재정위원회,예산결산특별위원회',
       '국회운영위원회,정무위원회', '법제사법위원회,예산결산특별위원회', '법제사법위원회,여성가족위원회',
       '국회운영위원회,산업통상자원중소벤처기업위원회', '기획재정위원회,여성가족위원회',
       '산업통상자원중소벤처기업위원회,여성가족위원회', '환경노동위원회,예산결산특별위원회',
       '농림축산식품해양수산위원회,예산결산특별위원회', '문화체육관광위원회,여성가족위원회',
       '과학기술정보방송통신위원회,예산결산특별위원회', '국방위원회

In [37]:
# 당선횟수2 라는 컬럼 생성

print(type(data_df['당선횟수']))
temp_str = data_df['당선횟수'].str  #(시리즈 형태를 str으로 변환)
print(type(temp_str))
numberof_election = temp_str[:2]
data_df['당선횟수2'] = numberof_election #(컬럼 추가)
data_df.loc[:,['당선횟수','당선횟수2']].head(20)

<class 'pandas.core.series.Series'>
<class 'pandas.core.strings.StringMethods'>


,당선횟수,당선횟수2
0,"재선(19대,21대)",재선
1,초선(21대),초선
2,초선(21대),초선
3,초선(21대),초선
4,초선(21대),초선
5,"재선(20대,21대)",재선
6,초선(21대),초선
7,초선(21대),초선
8,초선(21대),초선
9,"재선(20대,21대)",재선


In [38]:
data_df['당선횟수2'].value_counts()

초선    151
재선     74
3선     42
4선     19
5선     13
6선      1
Name: 당선횟수2, dtype: int64

In [41]:
data_df.loc[data_df['당선횟수2']=='6선']

,이름,이미지,생년월일,정당,선거구,소속위원회,당선횟수,사무실전화,홈페이지,이메일,보좌관,비서관,비서,"취미, 특기",당선횟수2
92,박병석,https://www.assembly.go.kr/photo/9770090.jpg,1952-01-25,무소속,대전서구갑,,"6선(16대,17대,18대,19대,20대,21대)",02-788-2306,,pbs2100@gmail.com,"강창희,송석근","예석해,최명식","김혜리,김민숙,유민선,심용진,최지연",,6선


In [43]:
data_df['생년월일'].head()

0    1960-06-04
1    1959-11-02
2    1963-05-27
3    1971-03-03
4    1961-04-26
Name: 생년월일, dtype: object

In [48]:
# Datatimeindex 클래스를 이용해서 생년월일 컬럼에서 년/월/일 값을 추출
# year, month, day 3개의 컬럼 추가

data_df['year'] = pd.DatetimeIndex(data_df['생년월일']).year
data_df['month'] = pd.DatetimeIndex(data_df['생년월일']).month
data_df['day'] = pd.DatetimeIndex(data_df['생년월일']).day
data_df.loc[:,['생년월일','year','month','day']].head()
# data_df.info()

,생년월일,year,month,day
0,1960-06-04,1960,6,4
1,1959-11-02,1959,11,2
2,1963-05-27,1963,5,27
3,1971-03-03,1971,3,3
4,1961-04-26,1961,4,26


In [60]:
# year 최대,최소 인 2명 누굴까요
# data_df['year'].min()
data_df.loc[data_df['year'] == 1992]

,이름,이미지,생년월일,정당,선거구,소속위원회,당선횟수,사무실전화,홈페이지,이메일,보좌관,비서관,비서,"취미, 특기",당선횟수2,year,month,day
80,류호정,https://www.assembly.go.kr/photo/9771274.jpg,1992-08-09,정의당,비례대표,산업통상자원중소벤처기업위원회,초선(21대),02-784-9740,http://ryuhojeong.com,help@ryuhojeong.com,"양호영,염종운","최강연,황윤영","이지은,김영미,임동준,손예지,김유일",게임,초선,1992,8,9


In [62]:
data_df.loc[data_df['year'] == 1947]
data_df.loc[data_df['year'] == data_df['year'].max(),['이름','정당','year']]

,이름,정당,year
80,류호정,정의당,1992


In [86]:
# 나이를 계산하는 함수 정의
from datetime import date

def calc_age(dtob):
    my_today =date.today()
    return my_today.year - dtob.year - ((my_today.month, my_today.day) < (dtob.month, dtob.day))

age_list = []
for idx, row in data_df.iterrows():
    age = calc_age(date(row['year'],row['month'],row['day']))
#     print(idx, age)
    age_list.append(age)
#     print(idx,row['year'],row['month'],'row[;day]')
print(len(age_list), age_list[0:4])

data_df['나이'] = age_list
data_df.loc[:,['이름','생년월일','나이']].head()
max_age = data_df['나이'].max()
min_age = data_df['나이'].min()
mean_age = data_df['나이'].mean()
print(max_age, min_age, mean_age)

300 [60, 60, 57, 49]
73 27 55.21


In [88]:
# 선거구2 컬럼을 추가

temp_str = data_df['선거구'].str
data_df['선거구2'] = temp_str[0:2]

data_df.loc[:,['선거구','선거구2']]
# data_df['선거구'].head()

,선거구,선거구2
0,경남창원시성산구,경남
1,대구동구을,대구
2,경기안양시만안구,경기
3,경남진주시을,경남
4,비례대표,비례
...,...,...
295,경기고양시병,경기
296,대구수성구을,대구
297,부산중구영도구,부산
298,대전중구,대전


In [92]:
data_df['선거구2'].value_counts()
data_df['선거구2'].value_counts(normalize=True)

경기    0.196667
서울    0.163333
비례    0.156667
부산    0.060000
경남    0.053333
경북    0.043333
인천    0.043333
대구    0.040000
충남    0.036667
전북    0.033333
전남    0.033333
강원    0.026667
충북    0.026667
광주    0.026667
대전    0.023333
울산    0.020000
제주    0.010000
세종    0.006667
Name: 선거구2, dtype: float64

In [93]:
data_df

,이름,이미지,생년월일,정당,선거구,소속위원회,당선횟수,사무실전화,홈페이지,이메일,보좌관,비서관,비서,"취미, 특기",당선횟수2,year,month,day,나이,선거구2
0,강기윤,https://www.assembly.go.kr/photo/9771230.jpg,1960-06-04,미래통합당,경남창원시성산구,보건복지위원회,"재선(19대,21대)",02-784-1751,http://blog.naver.com/ggotop,ggotop@naver.com,"강종길,김홍광","박응서,최광림","김영록,안효상,이유진,홍지형,김지훈",,재선,1960,6,4,60,경남
1,강대식,https://www.assembly.go.kr/photo/9771142.jpg,1959-11-02,미래통합당,대구동구을,국방위원회,초선(21대),,,,"박홍규,정운태",유진영,"박종원,박순권,김광연,김현정,송민욱",,초선,1959,11,2,60,대구
2,강득구,https://www.assembly.go.kr/photo/9771174.jpg,1963-05-27,더불어민주당,경기안양시만안구,교육위원회,초선(21대),02-784-2747~9,https://blog.naver.com/dulipapa,mainsail440@daum.net,유진우,"김도균,홍미하","최경순,김혜원,최기섭,조나연,오세령",,초선,1963,5,27,57,경기
3,강민국,https://www.assembly.go.kr/photo/9771233.jpg,1971-03-03,미래통합당,경남진주시을,정무위원회,초선(21대),,,,"강민승,박진우","신우성,오경훈","성환종,김민규,김오주,정재희,이승연",,초선,1971,3,3,49,경남
4,강민정,https://www.assembly.go.kr/photo/9771283.jpg,1961-04-26,열린민주당,비례대표,교육위원회,초선(21대),,,,전경원,"김민혜,윤호숙","김수안,김성용,민지홍,서지혜,이태경",,초선,1961,4,26,59,비례
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,홍정민,https://www.assembly.go.kr/photo/9771183.jpg,1978-11-24,더불어민주당,경기고양시병,"국회운영위원회,과학기술정보방송통신위원회",초선(21대),,,,"유지영,조성환","이용남,주영민","오미숙,한상득,윤한균,박희철,박지혜",,초선,1978,11,24,41,경기
296,홍준표,https://www.assembly.go.kr/photo/9771145.jpg,1954-12-05,무소속,대구수성구을,"국방위원회,예산결산특별위원회","5선(15대,16대,17대,18대,21대)","02-784-7610,02-6788-7481",https://blog.naver.com/jphong803,jphong803@naver.com,"김민준,이종헌","김윤환,손성호","양재유,김은영,고천석,권희연,김태형",,5선,1954,12,5,65,대구
297,황보승희,https://www.assembly.go.kr/photo/9771130.jpg,1976-08-05,미래통합당,부산중구영도구,"과학기술정보방송통신위원회,예산결산특별위원회",초선(21대),,,,"서덕교,이준우","고범욱,홍영민","최지현,정인호,홍주형,윤성환,서승환",,초선,1976,8,5,43,부산
298,황운하,https://www.assembly.go.kr/photo/9771161.jpg,1962-09-10,더불어민주당,대전중구,"산업통상자원중소벤처기업위원회,예산결산특별위원회",초선(21대),02-784-1935,,unhah0301@gmail.com,"박승규,방인호","장진섭,최형준","유혜정,홍승경,이제문,정윤주,정서원",,초선,1962,9,10,57,대전
